## Zajęcia 7: Bayes Classifier

Wszystkie zadania ćwiczeniowe należy rozwiązywać w języku Python w kopii Jupyter Notebook'a dla danych zajęć w wyznaczonych miejscach (komórki z komentarzem `# Solution`).

Nie należy usuwać komórek z treścią zadań.

Należy wyświetlać outputy przy pomocy `print`

#### Czym jest Bag of Words (BoW)?

#### Na czym polega "naiwność" klasyfikatora Bayesa?

# Kroki do zaimplementowania klasyfikatora Bayesa:
![alt text](bayes_steps.png "Title")

P(C|X) = P(C) * P(X|C)

P(C)

15 - tekstów
<br>
5 - spam
<br>
10 - niespam

P(spam) = 5/15 = 1/3

<br>
P(X|C)

Lubię borowiki i kurki.                                 NIE SPAM
<br>
Lubię borowiki, borowiki i jeszcze raz borowiki.        NIE SPAM
<br>
Poszedłem na zakupy.                                    NIE SPAM

P(borowiki|NIE SPAM) = 4 / 14

P(zakupy|NIE SPAM) = 1 / 14

P(X|NIE SPAM) = 4/14 * 1/14 * ...



100 tekstów ze spamem, każde ma po 8 słów (i w pozostałych 99 nie występuje słowo borowiki)
<br>
1                                                       SPAM
<br>
2                                                       SPAM
<br>
...
<br>
Kup kup nasze nasze borowiki borowiki (TANIO TANIO)!!!  SPAM
<br>
...
<br>
99                                                      SPAM
<br>
100                                                     SPAM

P(borowiki|SPAM) = 2 / 800

#### Inferencja (czyli dokonanie predykcji modelu) na zbiorze testowym

Dla zdania ze zbioru testowego dla każdego słowa, które znajduje się w słowniku, obliczamy wartości P(xi|C), gdzie wartość z licznika oraz mianiownika bazuje na danych ze zbioru treningowego.

Czyli np. dla zdania "Kup pyszne borowiki" jeżeli tylko słowa "kup" oraz "pyszne" występują w słowniku obliczamy P(X|C) = P(kup|C) * P(borowiki|C).


#### Zadanie 1

Zaimplementuj i wytrenuj klasyfikator Bayesa (bez korzystania z gotowych implementacji algorytmu) na danych treningowych z wyzwania:

https://amueval.pl/challenge/Spamcl4Ssificationsms/

A następnie wygeneruj predykcje dla zbioru testowego i dokonaj zgłoszenia na stronie wyzwania w zakładce "Add Submission".

W pliku out.tsv muszą znajdować się wartości 0 lub 1 oddzielone nowymi liniami (bez nagłówka).

Można spróbować z różnymi wartościami alpha oraz liczbą słów w słowniku (czyli w kroku P(X|C) możemy uwzględniać tylko np. 500 najczęściej występujących słów - nie gwarantuję że to pomoże).

Proszę pamiętać o redukcji obliczeń (np. liczbę słów w danej klasie można zapisać w zmiennej, a nie robić to za każdym razem).

#### Proszę podać nr indeksu przy wysyłaniu zadania na Teamsach

In [ ]:
# Solution

# NR INDEKSU: 481825

import pandas as pd
import re

# used for laplace smoothing
alpha=0.32
classes=[0,1]

def read_and_prepare_data(messages_file, labels_file):
    messages=pd.read_csv(messages_file, sep='\t', header=None)
    labels=pd.read_csv(labels_file, sep='\t', header=None)
    # combine messages and labels into one dataframe
    alldata=pd.concat([messages, labels], axis=1)
    alldata.columns=['message', 'label']
    # make all messages lowercase and remove punctuation
    alldata['message']=alldata['message'].str.lower()
    alldata['message']=alldata['message'].apply(lambda x: re.sub(r'[^\w\s]','',x))
    return alldata
    
def get_number_of_documents(data):
    return data.shape[0]

def get_number_of_documents_in_class(data, class_value):
    return data[data['label']==class_value].shape[0]

def calculate_prior_prob(data, class_value):
    return get_number_of_documents_in_class(data, class_value)/get_number_of_documents(data)

def create_vocabulary(data):
    vocabulary=set()
    for message in data['message']:
        words=message.split()
        for word in words:
            vocabulary.add(word)
    return vocabulary

def get_total_word_count_in_class(data, class_value):
    total_word_count=0
    for message in data[data['label']==class_value]['message']:
        words=message.split()
        total_word_count+=len(words)
    return total_word_count

# This function calculates the likelihood for all words in the given vocabulary for given class
def calculate_likelihood_for_vocab(data,vocab,class_value):
    likelihood={}
    total_word_count=total_word_count_for_classes[class_value]
    for word in vocab:
        word_apperances_count=0
        for message in data[data['label']==class_value]['message']:
            words=message.split()
            word_apperances_count+=words.count(word)
        likelihood[word]=(word_apperances_count+alpha)/(total_word_count+len_vocab)
    return likelihood

def probablity_of_sentence_for_class(sentence,class_value):
    likelihood = likelihood_not_spam if class_value == 0 else likelihood_spam
    # 1 is neutral element for multiplication
    prob=1
    words=sentence.split()
    for word in words:
        if(word in likelihood):
            prob*=likelihood[word]
        else:
            # if word is not in the vocabulary, we calculate probability as before but without word apperances in that class (it's seen for the first time)
            prob*=(alpha/(total_word_count_for_classes[class_value]+(alpha*len_vocab)))
    return prob

def calculate_posterior_prob_for_class(sentence,class_value):
    prior=prior_prob_for_classes[class_value]
    prob_for_words=probablity_of_sentence_for_class(sentence, class_value)
    return prior*prob_for_words

def classify(sentence):
    sentence=sentence.lower()
    sentence=re.sub(r'[^\w\s]','',sentence)
    prob_not_spam=calculate_posterior_prob_for_class(sentence, 0)
    prob_spam=calculate_posterior_prob_for_class(sentence, 1)
    # if both probabilities are equal, it is better to classify as not spam (we don't want to classify something more likely to non spam as spam)
    if(prob_not_spam==prob_spam):
        return 0
    elif(prob_not_spam>prob_spam):
        return 0
    else:
        return 1
    
data=read_and_prepare_data('spam-classification/train/in.tsv', 'spam-classification/train/expected.tsv')
vocab=create_vocabulary(data)
len_vocab=len(vocab)

# calculate total word count for both classes and store results in a dictionary
total_word_count_for_classes={
        0: get_total_word_count_in_class(data, 0),
        1: get_total_word_count_in_class(data, 1)
    }

# calculate prior probabilities for both classes and store them in a dictionary
prior_prob_for_classes={
        0: calculate_prior_prob(data, 0),
        1: calculate_prior_prob(data, 1)
    }

# calculate likelihood for all words in the vocabulary for both classes and store them in dictionaries
likelihood_not_spam=calculate_likelihood_for_vocab(data, vocab, 0)
likelihood_spam=calculate_likelihood_for_vocab(data, vocab, 1)


test_data=pd.read_csv('spam-classification/test/in.tsv', sep='\t', header=None)
result=pd.DataFrame()

for sentence in test_data[0]:
    result = pd.concat([result, pd.DataFrame([[classify(sentence)]] )], ignore_index=True)
    
result.to_csv('spam-classification/test/out.tsv', sep='\t', header=False, index=False)